# 基于Python的拖曳工况下后视镜视野仿真

## 基于Python的后视镜视野仿真方法
本节将详细介绍如何使用Python搭建一个拖车后视镜视野仿真工具。仿真分为几部分：几何建模（车辆、拖车、后视镜和驾驶员的位置姿态建模）、视线投射计算（模拟光线从驾驶员眼睛通过后视镜反射看到拖车后方的过程），以及结果可视化。我们将使用Python的科学计算和图形库，必要时集成OpenCV等计算机视觉库，实现3D视野的仿真和验证。

### 1. 仿真场景定义
首先，需要建立一个简化的3D场景模型。考虑到我们不依赖复杂的3D建模软件（如Blender）直接导入完整模型，我们采用参数化的简化模型描述车辆、后视镜和拖车：
+ **坐标系** ：建立右手坐标系，通常以牵引车（拖车头）为参考。本文采用汽车纵向为x轴（x轴向前，朝车辆行驶方向），横向为y轴（向左为正y），竖直为z轴（向上为正z）。为了方便计算，我们可将坐标系原点放在驾驶员眼睛位置下方地面位置。这样能简化后续视线计算，因为驾驶员眼睛成为观察点原点。
+ **驾驶员眼睛和坐姿**：需要定义驾驶员眼点相对于车辆的参数，比如高度（眼高），座椅位置等。根据人体工程，一般驾驶员眼高约1.1~1.3米（具体可依据SAE J941眼椭圆确定不同分位的眼点范围）。为仿真不同身高和坐姿，我们可以将眼点作为可变参数。本示例假定某一固定眼点高度，如1.2米，位于车辆左座靠近中线的位置。之后在扩展场景中，我们会考虑不同眼点的变化。
+ **后视镜参数** ：定义后视镜的位置和朝向，以及镜面尺寸和类型。假设我们以驾驶员侧外后视镜为例，其位置可用相对于驾驶员眼的坐标表示，如在驾驶员左侧约0.5米、前方0.2米、稍低0.1米处（由于侧后视镜通常位于车窗前柱附近，略比眼睛低一点）。镜面的朝向用法向量表示，即镜面法线垂直于镜面平面、指向反射光线进入眼睛的方向。镜面尺寸则用宽度w和高度h表示（例如镜面宽度0.2m、高度0.15m）。镜面曲率：对于平面镜，曲率半径无限大；若为凸面镜，可给定曲率半径R（例如典型汽车凸面镜R约1200~1500mm）。在仿真中，不同曲率主要影响视野角度：平面镜的视场角等于镜子物理尺寸产生的几何遮挡角，而凸面镜等效扩大了可视角。我们可以通过调整镜面曲率参数，或等效地调整投射算法，实现对凸面镜的模拟（后面详述实现方法）。
+ **拖车模型**：拖车简化为规则长方体或多边形棱柱。主要参数是宽度（例如2.4m）、高度（例如2.5m）、以及相对于牵引车的位置（拖车前端与牵引车后部的距离，即挂接长度，假设比如车尾到拖车头距离1m）。为简单起见，我们假定拖车正后方在牵引车正后方，例如拖车后部相对于驾驶员眼睛原点在x轴负方向若干米处。举例来说，牵引车长4m，拖车长6m，则拖车尾部可能在驾驶员眼后方约10m处。我们将重点关注拖车尾部的四个边缘点（左上、左下、右上、右下角）以及地面上的一些参考点，用于判断这些点在后视镜中的可见性。

使用Python字典定义参数(单位：m)。

In [ ]:
import numpy as np
# 眼点
eye_pos = np.array([0.0, 0.0, 1.2])
# 镜面 
left_mirror = {
    "left_up_pos": np.array([0.6, 0.8, 1.1]), # 左上角位置
    "width": 0.2,  # 镜面宽度
    "height": 0.15,  # 镜面高度
    "curvature_radius": None,  # 曲率半径
}
right_mirror = {
    "left_up_pos": np.array([0.6, -1.8, 1.1]),  # 左上角位置
    "width": 0.2,  # 镜面宽度
    "height": 0.15,  # 镜面高度
    "curvature_radius": None,  # 曲率半径
}
trailer = {
    "distance": 8.0,  # 拖车尾部相对驾驶员眼睛的距离（例如眼睛到拖车尾10m，这里简化为8m用于演示）
    "width": 2.4,  
    "height": 2.5,  
}